# Intro to Modeling

Plan ➔ Acquire ➔ Prepare ➔ Explore ➔ **Model** ➔ Deliver

Before modeling:

0. Split your data
1. Data preprocessing

The modeling "loop"

1. Create a model
    - algorithm + hyperparams
    - training data
1. Evaluate the model
1. Repeat

After a certain amount of time or repititions has passed:

1. Compare models
1. Evaluate on test

In [4]:
# We'll use sklearn's Dummy Classifier as a standin for other classification algorithms
# The sklearn API operates in the same way as our "real" models
import pandas as pd
from sklearn.dummy import DummyClassifier
from sklearn.metrics import classification_report
import acquire
import prepare

import warnings
warnings.filterwarnings('ignore')

## Data Split

In [18]:
df = acquire.get_titanic_data()
train, validate, test = prepare.split_data(df)


In [19]:
train.shape, validate.shape, test.shape

((498, 13), (214, 13), (179, 13))

In [20]:
X_train, y_train = train.drop(columns='survived'), train.survived
X_validate, y_validate = validate.drop(columns='survived'), validate.survived
X_test, y_test = test.drop(columns='survived'), test.survived

## Create our First Model

### Aside: Working with sklearn ML objects

1. Create the object
1. Fit the object on training data
1. Use the object (.score, .predict, .transform)

In [21]:
# 1. Create the object
model = DummyClassifier(strategy='constant', constant=1) # here, we're predicting everyone survives
# 2. Fit the object
model.fit(X_train, y_train)

DummyClassifier(constant=1, strategy='constant')

Ways we use sklearn classification models:

- `.score` gives us accuracy
- `.predict` lets us make predictions given a set of indep vars
- `.predict_proba` gives us the probability that each observation falls into each label
- some specific model types have additional properties

In [22]:
accuracy = round(model.score(X_train, y_train), 2)
print(f'Training accuracy: {accuracy}')

Training accuracy: 0.38


In [23]:
# TODO: view the accuracy on the validate split
model.score(X_validate, y_validate)

0.38317757009345793

In [24]:
model.predict(X_validate)

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1])

When using `.predict`, the values passed must be the same shape as the values used to fit!

In [25]:
# TODO: create a new column on the train dataframe that contains the models predictions
train['prediction'] = model.predict(X_train)

In [26]:
train.head()

,passenger_id,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,deck,embark_town,alone,prediction
583,583,0,1,male,36.0,0,0,40.1250,C,First,A,Cherbourg,1,1
165,165,1,3,male,9.0,0,2,20.5250,S,Third,NaN,Southampton,0,1
50,50,0,3,male,7.0,4,1,39.6875,S,Third,NaN,Southampton,0,1
259,259,1,2,female,50.0,0,1,26.0000,S,Second,NaN,Southampton,0,1
306,306,1,1,female,NaN,0,0,110.8833,C,First,NaN,Cherbourg,1,1


In [27]:
# use the column you just created and the actual values in the survived column
# to generate a classification report
print(classification_report(train.survived, train.prediction, zero_division=True))

              precision    recall  f1-score   support

           0       1.00      0.00      0.00       307
           1       0.38      1.00      0.55       191

    accuracy                           0.38       498
   macro avg       0.69      0.50      0.28       498
weighted avg       0.76      0.38      0.21       498



In [28]:
pd.DataFrame(classification_report(train.survived, train.prediction, output_dict=True)).transpose()

,precision,recall,f1-score,support
0,0.000000,0.000000,0.000000,307.000000
1,0.383534,1.000000,0.554427,191.000000
accuracy,0.383534,0.383534,0.383534,0.383534
macro avg,0.191767,0.500000,0.277213,498.000000
weighted avg,0.147098,0.383534,0.212642,498.000000


## More models

Now we'll make more models, one model is the unique combination of:

- algorithm
- hyperparameters
- training data

In [29]:
# What if we randomly guess?
model2 = DummyClassifier(strategy='uniform')

# TODO: fit the model on the training data
model2.fit(X_train, y_train)

# TODO: see how this model performs on train
accuracy = model2.score(X_train, y_train)
round(accuracy, 2)

0.5

In [30]:
# Let's make a 3rd model, using the most frequently occurring class in the target
model3 = DummyClassifier(strategy='most_frequent')
model3.fit(X_train, y_train)
accuracy = model3.score(X_train, y_train)
round(accuracy, 2)

0.62

What are we looking for when evaluating model performance?

- Is the model overfit? I.e. does it perform drastically better on the training data compared to the validate split
- How good or bad is the model, i.e. how does it perform
    - compared to the other models
    - compared to the baseline model

## Compare and Finalize

In [ ]:
# TODO: compare the performance of your models on the validate split


In [37]:
# TODO: find the performance of your best model on the test split
